In [1]:
#import dependencies

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder

In [3]:
#Read in csv files, create dataframes. This Cell is for macOS

tuition_df = pd.read_csv('Resources/College_Pricing_Trends_76-24.csv')
underemployed_df = pd.read_csv('Resources//Underemployed.csv')
unemployed_df = pd.read_csv('Resources//Unemployed.csv')
wages_df = pd.read_csv('Resources/Wages.csv')

In [4]:
#Read in csv files, create dataframes. This Cell is for Windows

#tuition_df = pd.read_csv('Resources\College_Pricing_Trends_76-24.csv')
#underemployed_df = pd.read_csv('Resources\\Underemployed.csv')
#unemployed_df = pd.read_csv('Resources\\Unemployed.csv')
#wages_df = pd.read_csv('Resources\Wages.csv')


In [5]:
# Create function to drop unwanted columns that contain the same string

def drop_colums(df, column_contains):
    df = df.loc[:, ~df.columns.str.contains(column_contains)]
    return df

In [6]:
# Apply function

underemployed_df = drop_colums(underemployed_df, 'Unnamed')
wages_df = drop_colums(wages_df, 'Unnamed')

In [7]:
# Create function to convert date columns to desired 'Year' format

def convert_date(df):    
    df['Date'] = pd.to_datetime(df['Date'])
    df_august = df[df['Date'].dt.month == 8]
    df_august = df_august.reset_index(drop=True)
    df_august['Date'] = df_august['Date'].dt.year
    df_auguest = df_august.rename(columns={'Date':'Year'})
    return df_august


In [8]:
# Run dfs through function

unemployed_df_clean = convert_date(unemployed_df)
underemployed_df_clean = convert_date(underemployed_df)


/var/folders/23/gj_hbqlx0plfs4t9bkt1_2jc0000gn/T/ipykernel_38305/3037807325.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/23/gj_hbqlx0plfs4t9bkt1_2jc0000gn/T/ipykernel_38305/3037807325.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [9]:
# Convert 'Date' column in wages_df

wages_df['Date'] = pd.to_datetime(wages_df['Date'])
wages_df['Date'] = wages_df['Date'].dt.year

/var/folders/23/gj_hbqlx0plfs4t9bkt1_2jc0000gn/T/ipykernel_38305/658603101.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  wages_df['Date'] = pd.to_datetime(wages_df['Date'])


In [10]:
# Convert 'Date Range' column in tuition_df
tuition_df['Academic Year'] = tuition_df['Academic Year'].apply(lambda x: '20' + x.split('-')[0] if int(x.split('-')[0]) < 50 else '19' + x.split('-')[0])

# Rename 'Acedemic Year' column to 'Year'
tuition_df_clean = tuition_df.rename(columns={'Academic Year':'Year'})

#convert 'Year' to datetime datatype
tuition_df_clean['Year'] = pd.to_datetime(tuition_df_clean['Year'])
tuition_df_clean['Year'] = tuition_df_clean['Year'].dt.year

#Drop all rows before '1990'
tuition_df_clean = tuition_df_clean[tuition_df_clean['Year'] >= 1990]

# Display result
tuition_df_clean

,Year,Private Nonprofit Four-Year,One-Year % Change,Public Four-Year,One-Year % Change.1,Public Two-Year,One-Year % Change.2,Private Nonprofit Four-Year.1,One-Year % Change.3,Public Four-Year.1,One-Year % Change.4
19,1990,"$21,850",2.30%,"$4,470",6.70%,"$2,130",2.90%,"$31,540",1.80%,"$11,860",1.90%
20,1991,"$22,030",0.80%,"$4,740",6.00%,"$2,630",23.50%,"$31,860",1.00%,"$12,240",3.20%
21,1992,"$22,780",3.40%,"$5,080",7.20%,"$2,440",-7.20%,"$32,760",2.80%,"$12,710",3.80%
22,1993,"$23,300",2.30%,"$5,380",5.90%,"$2,650",8.60%,"$33,440",2.10%,"$13,140",3.40%
23,1994,"$24,190",3.80%,"$5,590",3.90%,"$2,700",1.90%,"$34,050",1.80%,"$13,660",4.00%
24,1995,"$24,520",1.40%,"$5,640",0.90%,"$2,670",-1.10%,"$34,880",2.40%,"$13,530",-1.00%
25,1996,"$25,320",3.30%,"$5,810",3.00%,"$2,870",7.50%,"$35,790",2.60%,"$13,920",2.90%
26,1997,"$26,280",3.80%,"$5,930",2.10%,"$2,990",4.20%,"$36,890",3.10%,"$14,230",2.20%
27,1998,"$27,600",5.00%,"$6,100",2.90%,"$2,910",-2.70%,"$38,390",4.10%,"$14,580",2.50%
28,1999,"$28,490",3.20%,"$6,170",1.10%,"$3,030",4.10%,"$39,430",2.70%,"$14,830",1.70%


In [11]:
# Rename 'Date' to 'Year'

unemployed_df_clean = unemployed_df_clean.rename(columns={'Date':'Year'})
underemployed_df_clean = underemployed_df_clean.rename(columns={'Date':'Year'})
wages_df_clean = wages_df.rename(columns={'Date':'Year'})


In [12]:
display(underemployed_df_clean.head())
display(unemployed_df_clean.head())
display(wages_df_clean.head())
display(tuition_df_clean.head())

,Year,Recent graduates,College graduates
0,1990,43.8,33.5
1,1991,44.4,33.9
2,1992,47.3,34.7
3,1993,46.8,34.4
4,1994,46.5,33.7


,Year,Young workers,All workers,Recent graduates,College graduates
0,1990,7.9,5.3,3.7,2.4
1,1991,9.9,6.6,4.8,3.0
2,1992,11.0,7.4,4.9,3.3
3,1993,10.1,6.6,4.0,2.9
4,1994,9.0,5.8,4.3,2.7


,Year,Bachelor's degree: 25th percentile,Bachelor's degree: median,Bachelor's degree: 75th percentile,High school diploma: median
0,1990,"39,984","52,353","67,188","38,073"
1,1991,"37,393","49,427","64,470","35,673"
2,1992,"37,540","50,054","62,567","33,369"
3,1993,"36,457","48,610","62,788","34,230"
4,1994,"35,387","47,380","61,199","33,561"


,Year,Private Nonprofit Four-Year,One-Year % Change,Public Four-Year,One-Year % Change.1,Public Two-Year,One-Year % Change.2,Private Nonprofit Four-Year.1,One-Year % Change.3,Public Four-Year.1,One-Year % Change.4
19,1990,"$21,850",2.30%,"$4,470",6.70%,"$2,130",2.90%,"$31,540",1.80%,"$11,860",1.90%
20,1991,"$22,030",0.80%,"$4,740",6.00%,"$2,630",23.50%,"$31,860",1.00%,"$12,240",3.20%
21,1992,"$22,780",3.40%,"$5,080",7.20%,"$2,440",-7.20%,"$32,760",2.80%,"$12,710",3.80%
22,1993,"$23,300",2.30%,"$5,380",5.90%,"$2,650",8.60%,"$33,440",2.10%,"$13,140",3.40%
23,1994,"$24,190",3.80%,"$5,590",3.90%,"$2,700",1.90%,"$34,050",1.80%,"$13,660",4.00%


In [13]:
tuition_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 19 to 52
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Year                           34 non-null     int32 
 1   Private Nonprofit Four-Year    34 non-null     object
 2   One-Year % Change              34 non-null     object
 3   Public Four-Year               34 non-null     object
 4   One-Year % Change.1            34 non-null     object
 5   Public Two-Year                34 non-null     object
 6   One-Year % Change.2            34 non-null     object
 7   Private Nonprofit Four-Year.1  34 non-null     object
 8   One-Year % Change.3            34 non-null     object
 9   Public Four-Year.1             34 non-null     object
 10  One-Year % Change.4            34 non-null     object
dtypes: int32(1), object(10)
memory usage: 3.1+ KB


In [14]:
# Merge all dataframes on 'year' column
merged_df = underemployed_df_clean.merge(unemployed_df_clean, on='Year').merge(wages_df_clean, on='Year').merge(tuition_df_clean, on='Year')

# Display first 20 results
merged_df.head(20)

,Year,Recent graduates_x,College graduates_x,Young workers,All workers,Recent graduates_y,College graduates_y,Bachelor's degree: 25th percentile,Bachelor's degree: median,Bachelor's degree: 75th percentile,...,Private Nonprofit Four-Year,One-Year % Change,Public Four-Year,One-Year % Change.1,Public Two-Year,One-Year % Change.2,Private Nonprofit Four-Year.1,One-Year % Change.3,Public Four-Year.1,One-Year % Change.4
0,1990,43.8,33.5,7.9,5.3,3.7,2.4,"39,984","52,353","67,188",...,"$21,850",2.30%,"$4,470",6.70%,"$2,130",2.90%,"$31,540",1.80%,"$11,860",1.90%
1,1991,44.4,33.9,9.9,6.6,4.8,3.0,"37,393","49,427","64,470",...,"$22,030",0.80%,"$4,740",6.00%,"$2,630",23.50%,"$31,860",1.00%,"$12,240",3.20%
2,1992,47.3,34.7,11.0,7.4,4.9,3.3,"37,540","50,054","62,567",...,"$22,780",3.40%,"$5,080",7.20%,"$2,440",-7.20%,"$32,760",2.80%,"$12,710",3.80%
3,1993,46.8,34.4,10.1,6.6,4.0,2.9,"36,457","48,610","62,788",...,"$23,300",2.30%,"$5,380",5.90%,"$2,650",8.60%,"$33,440",2.10%,"$13,140",3.40%
4,1994,46.5,33.7,9.0,5.8,4.3,2.7,"35,387","47,380","61,199",...,"$24,190",3.80%,"$5,590",3.90%,"$2,700",1.90%,"$34,050",1.80%,"$13,660",4.00%
5,1995,44.4,32.9,8.5,5.4,3.5,2.6,"33,605","46,087","59,529",...,"$24,520",1.40%,"$5,640",0.90%,"$2,670",-1.10%,"$34,880",2.40%,"$13,530",-1.00%
6,1996,43.9,33.1,7.7,5.0,3.8,2.4,"33,579","46,638","61,562",...,"$25,320",3.30%,"$5,810",3.00%,"$2,870",7.50%,"$35,790",2.60%,"$13,920",2.90%
7,1997,40.8,32.5,7.6,4.6,3.0,2.1,"36,458","47,396","62,890",...,"$26,280",3.80%,"$5,930",2.10%,"$2,990",4.20%,"$36,890",3.10%,"$14,230",2.20%
8,1998,40.4,32.3,7.3,4.3,2.9,1.8,"35,903","50,264","64,086",...,"$27,600",5.00%,"$6,100",2.90%,"$2,910",-2.70%,"$38,390",4.10%,"$14,580",2.50%
9,1999,38.1,31.6,6.7,4.0,3.5,1.9,"37,767","50,942","68,510",...,"$28,490",3.20%,"$6,170",1.10%,"$3,030",4.10%,"$39,430",2.70%,"$14,830",1.70%
